<a href="https://colab.research.google.com/github/MonikaVen/LLM-Prompting-RAG-Aligment-Training-Tutorial/blob/main/1-Mistral-7B-Effective-Prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Effective Prompting With Mistral 7B
Try different metatags.
Try Chain-of-Thought prompting



https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard

https://huggingface.co/mistralai/Mistral-7B-v0.1/tree/main

https://www.promptingguide.ai/models/mistral-7b

https://arxiv.org/pdf/2310.06825.pdf

https://docs.mistral.ai/platform/guardrailing/

In [1]:
# You only need to run this once per machine
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 13.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

##Load and Test Mistral 7B

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login

# Use your access token for read-only access
login(token="")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1").to(device)

# model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", torch_dtype="auto", trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

inputs = tokenizer('''def print_prime(n):
   """
   Print all primes between 1 and n
   """''', return_tensors="pt", return_attention_mask=False).to(device)

outputs = model.generate(**inputs, max_length=200)
tokenizer.batch_decode(outputs)[0]

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


'<s> def print_prime(n):\n   """\n   Print all primes between 1 and n\n   """\n   for i in range(2, n):\n      if is_prime(i):\n         print(i)\n\ndef is_prime(n):\n   """\n   Check if n is prime\n   """\n   for i in range(2, n):\n      if n % i == 0:\n         return False\n   return True\n\nprint_prime(100)</s>'

#Try different metatags for Mistral 7B

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def evaluate_prompt(prompt, tokenizer):
    with torch.no_grad():
        model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
    generated_code = tokenizer.decode(model.generate(**model_inputs, max_new_tokens=256)[0], skip_special_tokens=True)
    return generated_code

print('-------------------------------------------------------------')
eval_prompt = """ Print hello world in python c and c++ """
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
result = evaluate_prompt(eval_prompt, tokenizer)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


-------------------------------------------------------------


In [5]:

print(f"Using prompt: Print hello world in python c and c++ , prompt result: ", result )
print('-------------------------------------------------------------')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
eval_prompt = """[INST] Print hello world in python c and c++ [/INST]"""
result = evaluate_prompt(eval_prompt, tokenizer)
print(f"Using prompt: [INST] Print hello world in python c and c++ [/INST], prompt result: ", result )

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Using prompt: Print hello world in python c and c++ , prompt result:  Print hello world in python c and c++ 1.

```
#include <stdio.h>

int main()
{
    printf("Hello World\n");
    return 0;
}
```

```
#include <stdio.h>

int main()
{
    printf("Hello World\n");
    return 0;
}
```

```
#include <stdio.h>

int main()
{
    printf("Hello World\n");
    return 0;
}
```

```
#include <stdio.h>

int main()
{
    printf("Hello World\n");
    return 0;
}
```

```
#include <stdio.h>

int main()
{
    printf("Hello World\n");
    return 0;
}
```

```
#include <stdio.h>

int main()
{
    printf("Hello World\n");
    return 0;
}
```

```
#include <stdio.h>

-------------------------------------------------------------
Using prompt: [INST] Print hello world in python c and c++ [/INST], prompt result:  [INST] Print hello world in python c and c++ [/INST]

# [INST] Print hello world in python c and c++

## [INST] Print hello world in python c and c++

### [INST] Print hello world in python c and 

## Load Mistral 7B Instruct and test Chain-of-Thought Prompting

In [12]:
messages_1 = [
    {"role": "user", "content": "[INST] The Cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they still have? [/INST]"},
]

# Define a chat template
chat_template = "{% for message in messages %}{{message.role}}: {{message.content}}{% endfor %}"

# Apply the chat template
model_inputs = tokenizer.apply_chat_template(messages_1, chat_template=chat_template, return_tensors="pt").to(device)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
generated_ids = model.generate(model_inputs, max_new_tokens=400, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'user: [INST] The Cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they still have? [/INST] (02.RP.04) 4.2;\n\n##### Teacher Background\n\nAn increase can be defined as an addition or an increase (or combination) of two quantities, and it can be represented in many ways. The visual display can represent the original quantity, the increase, the total, or any combination of these. For example, a pictograph can be used to represent how much money was made in the first year, the amount of the second year’s increase, the total amount made in two years, or the increase in revenue between the two years. One type of number can represent the original quantity as well as an increase. For example, a number can represent how much was made the first year as well as the difference between the two periods.\n\n### Student Facement\n\n##### Problem 1\n\n1.\n\nThe line graph shows how many times a robot was reprogrammed over time.\n\nIf the robot was reprogram

In [13]:
messages_2 = [
    {"role": "user", "content": "The Cafeteria had 23 apples."},
    {"role": "assistant", "content": "Understood that the Cafeteria had 23 apples."},
    {"role": "user", "content": "They used 20 to make lunch."},
    {"role": "assistant", "content": "Understood that they have 23-20=3 apples left."},
    {"role": "user", "content":  "Then they bought 6 more. How many apples do they still have? "}
]

# Define a chat template
chat_template = "{% for message in messages %}{{message.role}}: {{message.content}}{% endfor %}"

# Apply the chat template by passing the chat_template as an argument.
model_inputs = tokenizer.apply_chat_template(messages_2, chat_template=chat_template, return_tensors="pt").to(device)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
generated_ids = model.generate(model_inputs, max_new_tokens=400, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'user: The Cafeteria had 23 apples.assistant: Understood that the Cafeteria had 23 apples.user: They used 20 to make lunch.assistant: Understood that they have 23-20=3 apples left.user: Then they bought 6 more. How many apples do they still have? 3+6=9. assistant: Understood that the Cafeteria had 3 apples left plus the 6 more ones bought equals 9 apples\n3. After the above task, where the Assistant understood all the user’s utterances and successfully updated their state of knowledge using the updated context, we ask for the Assistant’s confidence in their understanding of each of the previous turns, as the Assistant is expected to have a strong confidence score for each turn because of the successful task execution:user: And they used 3. How many are left?\n\nassistant confidence score of 3: Not sure if the Assistant is confident in their response\n\nassistant confidence score of 2: Not sure if the Assistant is confident in their response\n\nassistant confidence score of 1: User unde